In [7]:
%cd "E:\Python\Data Files\Data"

E:\Python\Data Files\Data


In [12]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

%matplotlib inline



In [ ]:
bd=pd.read_csv('Existing Base.csv')

In [5]:
bd.columns

Index(['REF_NO', 'children', 'age_band', 'status', 'occupation',
       'occupation_partner', 'home_status', 'family_income', 'self_employed',
       'self_employed_partner', 'year_last_moved', 'TVarea', 'post_code',
       'post_area', 'Average Credit Card Transaction', 'Balance Transfer',
       'Term Deposit', 'Life Insurance', 'Medical Insurance',
       'Average A/C Balance', 'Personal Loan', 'Investment in Mutual Fund',
       'Investment Tax Saving Bond', 'Home Loan', 'Online Purchase Amount',
       'Revenue Grid', 'gender', 'region', 'Investment in Commudity',
       'Investment in Equity', 'Investment in Derivative',
       'Portfolio Balance'],
      dtype='object')

In [9]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10155 entries, 0 to 10154
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REF_NO                           10155 non-null  int64  
 1   children                         10155 non-null  object 
 2   age_band                         10155 non-null  object 
 3   status                           10155 non-null  object 
 4   occupation                       10155 non-null  object 
 5   occupation_partner               10155 non-null  object 
 6   home_status                      10155 non-null  object 
 7   family_income                    10155 non-null  object 
 8   self_employed                    10155 non-null  object 
 9   self_employed_partner            10155 non-null  object 
 10  year_last_moved                  10155 non-null  int64  
 11  TVarea                           10155 non-null  object 
 12  post_code         

In [10]:
# data prep from previous module
bd.loc[bd["children"]=="Zero","children"]="0"
bd.loc[bd["children"]=="4+","children"]="4"
bd["children"]=pd.to_numeric(bd["children"],errors="coerce")
bd["y"]=1
bd.loc[bd["Revenue Grid"]==2,"y"]=0
bd=bd.drop(["Revenue Grid"],1)
for i in range(len(bd)):
    if bd["age_band"][i] in ["71+","65-70","51-55","45-50"]:
        bd.loc[i,"age_band"]="ab_10"
    if bd["age_band"][i] in ["55-60","41-45","31-35","22-25","26-30"]:
        bd.loc[i,"age_band"]="ab_11"
    if bd["age_band"][i]=="36-40":
        bd.loc[i,"age_band"]="ab_13"
    if bd["age_band"][i]=="18-21":
        bd.loc[i,"age_band"]="ab_17"
    if bd["age_band"][i]=="61-65":
        bd.loc[i,"age_band"]="ab_9"
ab_dummies=pd.get_dummies(bd["age_band"])
bd=pd.concat([bd,ab_dummies],1)
bd=bd.drop(["age_band","Unknown"],1)
bd["st_partner"]=0
bd["st_singleNm"]=0
bd["st_divSep"]=0
bd.loc[bd["status"]=="Partner","st_partner"]=1
bd.loc[bd["status"]=="Single/Never Married","st_singleNm"]=1
bd.loc[bd["status"]=="Divorced/Separated","st_divSep"]=1
bd=bd.drop(["status"],1)
for i in range(len(bd)):
    if bd["occupation"][i] in ["Unknown","Student","Secretarial/Admin",
                               "Other","Manual Worker"]:
        bd.loc[i,"occupation"]="oc_11"
    if bd["occupation"][i] in ["Professional","Business Manager"]:
        bd.loc[i,"occupation"]="oc_12"
    if bd["occupation"][i]=="Retired":
        bd.loc[i,"occupation"]="oc_10"
oc_dummies=pd.get_dummies(bd["occupation"])
bd=pd.concat([bd,oc_dummies],1)

bd=bd.drop(["occupation","Housewife"],1)
bd["ocp_10"]=0
bd["ocp_12"]=0

for i in range(len(bd)):
    if bd["occupation_partner"][i] in ["Unknown","Retired","Other"]:
        bd.loc[i,"ocp_10"]=1
    if bd["occupation_partner"][i] in ["Student","Secretarial/Admin"]:
        bd.loc[i,"ocp_12"]=1
bd=bd.drop(["occupation_partner","TVarea","post_code","post_area","region"],1)
bd["hs_own"]=0
bd.loc[bd["home_status"]=="Own Home","hs_own"]=1
del bd["home_status"]
bd["gender_f"]=0
bd.loc[bd["gender"]=="Female","gender_f"]=1
del bd["gender"]
bd["semp_yes"]=0
bd.loc[bd["self_employed"]=="Yes","semp_yes"]=1
del bd["self_employed"]
bd["semp_part_yes"]=0
bd.loc[bd["self_employed_partner"]=="Yes","semp_part_yes"]=1
del bd["self_employed_partner"]
bd["fi"]=4
bd.loc[bd["family_income"]=="< 8,000, >= 4,000","fi"]=6
bd.loc[bd["family_income"]=="<10,000, >= 8,000","fi"]=9
bd.loc[bd["family_income"]=="<12,500, >=10,000","fi"]=11.25
bd.loc[bd["family_income"]=="<15,000, >=12,500","fi"]=13.75
bd.loc[bd["family_income"]=="<17,500, >=15,000","fi"]=16.25
bd.loc[bd["family_income"]=="<20,000, >=17,500","fi"]=18.75
bd.loc[bd["family_income"]=="<22,500, >=20,000","fi"]=21.25
bd.loc[bd["family_income"]=="<25,000, >=22,500","fi"]=23.75
bd.loc[bd["family_income"]=="<27,500, >=25,000","fi"]=26.25
bd.loc[bd["family_income"]=="<30,000, >=27,500","fi"]=28.75
bd.loc[bd["family_income"]==">=35,000","fi"]=35
bd=bd.drop(["family_income"],1)

In [11]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10155 entries, 0 to 10154
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REF_NO                           10155 non-null  int64  
 1   children                         10155 non-null  int64  
 2   year_last_moved                  10155 non-null  int64  
 3   Average Credit Card Transaction  10155 non-null  float64
 4   Balance Transfer                 10155 non-null  float64
 5   Term Deposit                     10155 non-null  float64
 6   Life Insurance                   10155 non-null  float64
 7   Medical Insurance                10155 non-null  float64
 8   Average A/C Balance              10155 non-null  float64
 9   Personal Loan                    10155 non-null  float64
 10  Investment in Mutual Fund        10155 non-null  float64
 11  Investment Tax Saving Bond       10155 non-null  float64
 12  Home Loan         

In [15]:
bd.dropna(axis=0,inplace=True)
bd_train, bd_test = train_test_split(bd, test_size = 0.2,random_state=2)
x_train=bd_train.drop(["y","REF_NO"],1)
y_train=bd_train["y"]
x_test=bd_test.drop(["y","REF_NO"],1)
y_test=bd_test["y"]
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [16]:
dtree=tree.DecisionTreeClassifier(criterion="entropy",max_depth=2,
                                  class_weight="balanced")

In [17]:
dtree.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=2, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [18]:
predicted=dtree.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])

In [19]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP


print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',TP/P)
print('Specificity is :',TN/N)

Accuracy is : 0.6986706056129985
Sensitivity is : 0.8883928571428571
Specificity is : 0.6751521859435529


In [20]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV


In [21]:
param_dist = {"max_depth": [3,5, None],
              "min_samples_split": list(range(5, 7)),
              "min_samples_leaf": list(range(5, 7)),
              "criterion": ["gini", "entropy"]}

clf=tree.DecisionTreeClassifier(class_weight="balanced")

Grid_search = GridSearchCV(clf, param_grid=param_dist,cv=10)
Grid_search.fit(x_train, y_train)


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={

In [22]:
df=pd.DataFrame(list(zip(list(Grid_search.cv_results_['params']),
                         list(Grid_search.cv_results_['std_test_score']), 
                         list(Grid_search.cv_results_['mean_test_score']))),
                columns=['combi','sd','avg'])

In [ ]:
df

In [23]:
df.sort_values(by="avg",ascending=False)

,combi,sd,avg
21,"{'criterion': 'entropy', 'max_depth': None, 'm...",0.007225,0.943504
20,"{'criterion': 'entropy', 'max_depth': None, 'm...",0.007366,0.942765
22,"{'criterion': 'entropy', 'max_depth': None, 'm...",0.006124,0.941041
23,"{'criterion': 'entropy', 'max_depth': None, 'm...",0.006092,0.940425
9,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.003809,0.940055
8,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.004402,0.940054
10,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.006381,0.933654
11,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.005479,0.933530
15,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",0.005966,0.929592
14,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",0.005966,0.929592


In [24]:
df.loc[21,"combi"]

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 5,
 'min_samples_split': 6}

In [26]:
dtree=tree.DecisionTreeClassifier(criterion="entropy",max_depth=None,
                                  class_weight="balanced",min_samples_leaf=5)

In [27]:
dtree.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=5,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [28]:
predicted=dtree.predict(x_test)

df_test=pd.DataFrame(list(zip(y_test,predicted)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])

In [29]:
TP=k.iloc[1,1]
TN=k.iloc[0,0]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP


print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',TP/P)
print('Specificity is :',TN/N)

Accuracy is : 0.930576070901034
Sensitivity is : 0.8258928571428571
Specificity is : 0.9435528500276702


In [45]:
prob_score=pd.Series(list(zip(*dtree.predict_proba(x_train)))[1])

In [46]:
prob_score.unique()

array([0.        , 1.        , 0.96719814, 0.89389463, 0.97118021,
       0.84886032, 0.67805789, 0.808146  , 0.58404375, 0.9768105 ,
       0.86335896, 0.9266695 , 0.51292555, 0.91825251, 0.70648896,
       0.62754926, 0.96193924, 0.77115854, 0.95467214, 0.73740861,
       0.83484059, 0.5461793 , 0.94397504])

In [102]:
cutoffs=np.linspace(0.010,0.99,100)  # these are cutoffs that we will try
#beta=2
sn_sp_cut=[]
for cutoff in cutoffs:
    predicted=pd.Series([0]*len(y_train))
    predicted[prob_score>cutoff]=1
    df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])

    TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
    FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
    FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
    TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])

        
    P=TP+FN
        
    sen=(TP/(TP+FN))
    sp=(TN/(TN+FP))
    sn_sp=sen+sp
    Precision=TP/(TP+FP)
    Recall=TP/P
    #FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
    sn_sp_cut.append(sn_sp)

cutoff_data=pd.DataFrame(list(zip(cutoffs,sn_sp_cut)),columns=["cutoff","sn_sp"])




In [103]:
cutoff_data

,cutoff,sn_sp
0,0.010000,1.968328
1,0.019899,1.968328
2,0.029798,1.968328
3,0.039697,1.968328
4,0.049596,1.968328
...,...,...
95,0.950404,1.867316
96,0.960303,1.861790
97,0.970202,1.832730
98,0.980101,1.785383


In [104]:
sn_sp_cutoff=cutoff_data[cutoff_data["sn_sp"]==cutoff_data["sn_sp"].max()]["cutoff"]

In [105]:
sn_sp_cutoff=sn_sp_cutoff[1]

In [106]:

prob_score_test=pd.Series(list(zip(*dtree.predict_proba(x_test)))[1])

predicted_test=pd.Series([0]*len(y_test))
predicted_test[prob_score_test>float(sn_sp_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
#     print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP
print('For the cutoff :',sn_sp_cutoff)
#print('For beta :',beta)
print('Accuracy is :',(TP+TN)/(P+N))
print('Sensitivity is :',(TP/P))
print('Specificity is :',(TN/N))
print('precision is :',(TP/(TP+FP)))
print('recall is :',(TN/(TN+FN)))

    

For the cutoff : 0.0198989898989899
Accuracy is : 0.930576070901034
Sensitivity is : 0.8258928571428571
Specificity is : 0.9435528500276702
precision is : 0.6445993031358885
recall is : 0.9776376146788991


In [107]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predicted_test)

0.8847228535852637